In [56]:
import json
import os
from tqdm import tqdm
import plotly.graph_objects as go
from collections import OrderedDict

In [59]:
in_folder_path = '../../results/v2_out_2'

In [65]:
modal_splits = {}

for f_name in tqdm(os.listdir(in_folder_path)):
    if f_name.startswith('ms'):
        modal_name = f_name.replace('.json', '')

        with open(in_folder_path+'/'+f_name, 'r') as f:
            ms = json.load(f)

        drivers_mean = ms['car_driver']['median']
        passengers_mean = ms['car_passenger']['median']
        drivers_std = ms['car_driver']['std']
        passengers_std = ms['car_passenger']['std']
        
        ms['car'] = {}
        s = drivers_mean + passengers_mean
        ms['car']['median'] = s
        ms['car']['std'] = (((drivers_mean / s) * drivers_std) + ((passengers_mean / s) * passengers_std)) / 2

        del ms['car_driver']
        del ms['car_passenger']

        
        ms = OrderedDict((k, ms[k]) for k in [
            'car', 'public', 'pedestrian', 'bicycle'
        ])

        modal_splits[modal_name] = ms


100%|██████████| 7/7 [00:00<00:00, 1525.99it/s]


## Grouped bar plots

### Household_cars

In [66]:
modes=list(list(modal_splits.values())[0].keys())

mean_0_0_0 = modal_splits['ms_scenario_0_0_0'].values()
mean_0_0_0 = list(map(lambda x: x['median'], mean_0_0_0))

mean_0_0_15 = modal_splits['ms_scenario_0_0_15'].values()
mean_0_0_15 = list(map(lambda x: x['median'], mean_0_0_15))

mean_0_0_3 = modal_splits['ms_scenario_0_0_3'].values()
mean_0_0_3 = list(map(lambda x: x['median'], mean_0_0_3))

fig = go.Figure(data=[
    go.Bar(name='0', x=modes, y=mean_0_0_0),
    go.Bar(name='15', x=modes, y=mean_0_0_15),
    go.Bar(name='3', x=modes, y=mean_0_0_3)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [117]:
scenarios = ['30%', '15%', '0%']

mean_0_0_0 = modal_splits['ms_scenario_0_0_0'].values()
mean_0_0_0 = list(map(lambda x: x['median'], mean_0_0_0))

mean_0_0_15 = modal_splits['ms_scenario_0_0_15'].values()
mean_0_0_15 = list(map(lambda x: x['median'], mean_0_0_15))

mean_0_0_3 = modal_splits['ms_scenario_0_0_3'].values()
mean_0_0_3 = list(map(lambda x: x['median'], mean_0_0_3))

fig = go.Figure()

round_v = 1
margin_size = 100

# car
y = scenarios
x = [mean_0_0_3[0], mean_0_0_15[0], mean_0_0_0[0]]
fig.add_trace(go.Bar(
    y=y,
    x=x,
    name='Samochód',
    orientation='h',
    text=list(map(lambda v: str(round(v, round_v)) + "%", x)),
    textposition='auto',
    insidetextanchor='middle'
))

# publlic
y = scenarios
x = [mean_0_0_3[1], mean_0_0_15[1], mean_0_0_0[1]]
fig.add_trace(go.Bar(
    y=y,
    x=x,
    name='Komunikacja publiczna',
    orientation='h',
    text=list(map(lambda v: str(round(v, round_v))+"%", x)),
    textposition='auto',
    insidetextanchor='middle'
))

# pedestrian
y = scenarios
x = [mean_0_0_3[2], mean_0_0_15[2], mean_0_0_0[2]]
fig.add_trace(go.Bar(
    y=y,
    x=x,
    name='Pieszo',
    orientation='h',
    text=list(map(lambda v: str(round(v, round_v)) + "%", x)),
    textposition='auto',
    insidetextanchor='middle'
))

# bicycle
y = scenarios
x = [mean_0_0_3[3], mean_0_0_15[3], mean_0_0_0[3]]
fig.add_trace(go.Bar(
    y=y,
    x=x,
    name='Rower',
    orientation='h',
    text=list(map(lambda v: str(round(v, round_v)) + "%", x)),
    textposition='auto',
    insidetextanchor='middle'
))

# Change the bar mode
fig.update_layout(
    barmode='stack',
    legend={
        'traceorder':'normal',
        'orientation': 'h',
        'yanchor': 'top',
        'y': 1.15,
        'xanchor': 'left',
        'x': 0.01
    },
    # title='Rozkład podróży na środki transportu dla scenariusza ...',
    xaxis_title='Mediana udziału środka transportu we wszystkich podróżach',
    yaxis_title='Odsetek mieszkańców rezygnujących <br> z jednego samochodu',
    margin=dict(l=margin_size, r=margin_size, t=margin_size, b=margin_size),
)
fig.show()